In [ ]:
!pip install google-adk --quiet


In [ ]:
import os

os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"  
os.environ["GOOGLE_API_KEY"] = "YOUR_API_KEY_HERE"  


In [ ]:
# Simple in-memory thread storage
THREAD_STATE = {}
DRAFTS = {}

def save_thread_state(thread_id, state):
    THREAD_STATE[thread_id] = state

def load_thread_state(thread_id):
    return THREAD_STATE.get(thread_id)

def save_draft(thread_id, draft_text):
    DRAFTS[thread_id] = draft_text

def load_draft(thread_id):
    return DRAFTS.get(thread_id)


In [ ]:
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def save_email_thread(thread_id: str, email_subject: str, email_body: str, triage_summary: str):
    logger.info(f"[Tool save_email_thread] thread_id={thread_id}")

    state = {
        "thread_id": thread_id,
        "last_subject": email_subject,
        "last_body": email_body,
        "triage_summary": triage_summary,
    }
    save_thread_state(thread_id, state)

    return {"status": "success", "thread_id": thread_id}


def load_thread_history(thread_id: str):
    logger.info(f"[Tool load_thread_history] thread_id={thread_id}")

    state = load_thread_state(thread_id)
    if state is None:
        return {"status": "not_found", "message": "No history for this thread_id"}

    return {"status": "success", "state": state}


def save_draft_tool(thread_id: str, draft_text: str):
    logger.info(f"[Tool save_draft_tool] thread_id={thread_id}, draft_len={len(draft_text)}")

    save_draft(thread_id, draft_text)

    return {
        "status": "success",
        "thread_id": thread_id,
        "draft_length": len(draft_text),
    }


In [ ]:
from google.adk.agents.llm_agent import Agent

root_agent = Agent(
    name="enterprise_email_triage_agent",
    model="gemini-2.0-flash",
    description="An enterprise agent for triage + drafting email responses.",
    instruction=(
        "You are an Enterprise Email Triage & Response Assistant.\n\n"
        "The user will pass:\n"
        "- email_subject\n"
        "- email_body\n"
        "- optional: thread_id\n\n"
        "YOUR TASKS:\n"
        "1) TRIAGE the email:\n"
        "   intent = [support, sales, scheduling, escalation, other]\n"
        "   priority = [low, medium, high]\n"
        "   human_review_recommended = true/false\n\n"
        "2) If thread_id provided:\n"
        "   * call load_thread_history\n"
        "   * after triage, call save_email_thread\n"
        "   * after drafting reply, call save_draft_tool\n\n"
        "3) Write a professional enterprise-grade reply.\n\n"
        "OUTPUT FORMAT:\n"
        "First give a 1–3 sentence triage explanation.\n"
        "Then output: 'DRAFT REPLY:' followed by the email reply.\n"
    ),
    tools=[
        save_email_thread,
        load_thread_history,
        save_draft_tool,
    ],
)


In [ ]:
def run_email_agent(email_subject, email_body, thread_id=None):
    prompt = {
        "email_subject": email_subject,
        "email_body": email_body,
    }

    if thread_id:
        prompt["thread_id"] = thread_id

    response = root_agent.run(prompt)
    return response


In [ ]:
response = run_email_agent(
    email_subject="Issue with my order #12345",
    email_body="Hello, I received the wrong item. Please help. This is urgent.",
    thread_id="order-12345"
)

response
